In [1]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import fitz
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document

In [2]:
# Directorio para cargar los archivos PDF
pdf_directory = './doc'  # Ajusta la ruta según tu estructura de directorios
os.makedirs(pdf_directory, exist_ok=True)

# Función para verificar si un archivo es PDF
def is_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            # Verificar si el archivo empieza con la cabecera %PDF
            header = file.read(4)
            if header != b'%PDF':
                return False

        # Intentar abrir el archivo con PyMuPDF
        doc = fitz.open(file_path)
        doc.close()
        return True
    except (fitz.FitzError, IOError):
        return False

# Directorio de entrada que contiene archivos PDF
input_directory = './doc'  # Cambia esta ruta si es diferente

# Mover y validar los archivos cargados
valid_pdf_files = []
for filename in os.listdir(input_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(input_directory, filename)
        print(f"Cargando archivo: {file_path}")

        # Mover el archivo al directorio especificado
        destination_path = os.path.join(pdf_directory, filename)

        # Mover el archivo al directorio especificado
        os.rename(file_path, destination_path)

        # Validar si es un PDF
        if is_pdf(destination_path):
            valid_pdf_files.append(filename)
        else:
            print(f"{filename} no es un archivo PDF válido y será ignorado.")

# Imprimir archivos válidos
print("Archivos PDF válidos:", valid_pdf_files)

Cargando archivo: ./doc\teoria.pdf
Archivos PDF válidos: ['teoria.pdf']


In [3]:
# Directorio que contiene los archivos PDF
pdf_directory = './doc'

# Lista para almacenar todos los documentos
all_docs = []

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        print(f"Cargando archivo: {file_path}")

        try:
            # Cargar el PDF
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            # Agregar los documentos cargados a la lista
            all_docs.extend(docs)
            print(f"Archivo {filename} cargado exitosamente.")
        except Exception as e:
            print(f"Error al cargar {filename}: {e}")

# Imprimir la cantidad total de partes del PDF
print(f"Cantidad total de partes del PDF: {len(all_docs)}")

Cargando archivo: ./doc\teoria.pdf
Archivo teoria.pdf cargado exitosamente.
Cantidad total de partes del PDF: 45


In [4]:
all_docs

[Document(metadata={'source': './doc\\teoria.pdf', 'page': 0}, page_content='1\nPLAN DE \nEMPRESA\nGuía para la elaboración del Plan de Empresa\n'),
 Document(metadata={'source': './doc\\teoria.pdf', 'page': 1}, page_content='Documento redactado por:\nAntonio Valentin Brito Cabrera\nRevisión de textos:\nElisa Rodríguez Pérez\nEstá prohibida su reproducción total o parcial, su \ntraducción, inclusión, transmisión, transformación, \nalmacenamiento o acceso a través de medios \nanalógicos, digitales o de cualquier otro sistema o \ntecnología creada o por crearse, sin autorización. \nCopyright ©. Todos los derechos reservados. \nMOSAECO, SL'),
 Document(metadata={'source': './doc\\teoria.pdf', 'page': 2}, page_content='ÍNDICE\n1\n3\n5\n12\n22\n26\n311. PRESENTACIÓN\n2. ACTIVIDAD Y PRODUCTO\n3. SECTOR Y MERCADO\n4. MARKETING Y VENTAS\n5. PRODUCCIÓN Y OPERACIONES\n6. RECURSOS HUMANOS\n7. ECONÓMICO Y FINANCIERO'),
 Document(metadata={'source': './doc\\teoria.pdf', 'page': 3}, page_content='1\

In [ ]:
# # Crear el modelo de embeddings
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# # Crear el Semantic Chunker
# semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

# # Dividir los documentos en partes semánticas manejables
# splits = []
# for doc in all_docs:
#     chunks = semantic_chunker.split_text(doc.page_content)
#     for chunk in chunks:
#         # Crear un objeto Document para cada fragmento
#         splits.append(Document(page_content=chunk, metadata=doc.metadata))

# # Crear el vector store y almacenar los vectores
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

# # Configurar el recuperador
# retriever = vectorstore.as_retriever()

# print("Vector store y recuperador creados exitosamente.")

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(all_docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

retriever = vectorstore.as_retriever()

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, a

In [ ]:
all_docs

In [11]:
retriever.get_relevant_documents('que es el QUÉ ES EL PLAN DE EMPRESA?')

[Document(metadata={'page': 0, 'source': './doc\\teoria.pdf'}, page_content='1\nPLAN DE \nEMPRESA\nGuía para la elaboración del Plan de Empresa'),
 Document(metadata={'page': 44, 'source': './doc\\teoria.pdf'}, page_content='42\nPLAN DE \nEMPRESA\nGuía para la elaboración del Plan de Empresa'),
 Document(metadata={'page': 3, 'source': './doc\\teoria.pdf'}, page_content='Este documento puede adoptar distintas formas. Los hay extensos \ny detallados. Los hay concisos y breves. De hecho, no existe ningún \nmodelo concreto de Plan de Empresa.\nCada persona emprendedora debería crear su propio plan, acorde \na su proyecto empresarial.\nEl Plan de Empresa es una herramienta de utilidad a la hora de poner \nen marcha un negocio. No obstante, un plan muy bien acabado no \ngarantiza el éxito aunque lo contrario tampoco es cierto.1.2. QUÉ ES EL PLAN DE EMPRESA'),
 Document(metadata={'page': 4, 'source': './doc\\teoria.pdf'}, page_content='socios/as.\nEl Plan de Empresa es una herramienta de mark

In [13]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [12]:
llm = ChatOllama(model='llama3.1:latest', temperature=0)

In [14]:
system_prompt = (
    "Eres un asistente virtual encargado de responder preguntas apoyándote en el contexto que tienes de los documentos pdf."
    "Tienes que responder fielmente con información de los documentos proporcionados."
    "En caso de que la pregunta no te da el contexto necesario, responde pidiendo lo que necesitas para poder contestarla."
    "Simpre responde con la informacíon del contexto. No respondas que no tienes acceso a la informacíon."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


chain = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(retriever, chain)

In [19]:
results = rag.invoke({"input": "que es el Público Objetivo?"})
results

{'input': 'que es el Público Objetivo?',
 'context': [Document(metadata={'page': 22, 'source': './doc\\teoria.pdf'}, page_content='posicionamiento son las siguientes: \n— Condicionan el comportamiento de compra del público objetivo. \n— Facilitan la diferenciación con respecto a la competencia. \nA partir de ellas se pueden desarrollar ventajas competitivas \nsostenibles al estar ligadas a puntos fuertes del negocio. 4.6. PLAN DE VENTAS'),
  Document(metadata={'page': 21, 'source': './doc\\teoria.pdf'}, page_content='19\nPara ello, el mensaje tendrá formato de información o anuncio y \nse transmitirá a través de diferentes medios tradicionales como  \ntelevisión, prensa y radio o digitales como las páginas web o redes \nsociales.  \nb. Promoción de ventas\nEl objetivo de la promoción de ventas es incrementar en el corto \nplazo las compras. Para ello, el mensaje se transmitirá mediante \nexpositores y otros soportes, principalmente en el punto de venta, \nofreciendo descuentos en preci

In [20]:
results['answer']

'Según el contexto proporcionado en los documentos pdf, el Público Objetivo se refiere a la audiencia o grupo de personas que la empresa está dirigiendo sus mensajes y acciones de marketing hacia. En otras palabras, son las personas a quienes se pretende influir para que realicen compras o adopten ciertas actitudes.\n\nEn concreto, en el documento pdf se menciona que el objetivo de las Relaciones Públicas es "mejorar y crear los contactos, opiniones y actitud favorable del público objetivo hacia la empresa, sus productos y servicios".\n\nPor lo tanto, el Público Objetivo es un concepto clave en marketing y comunicación que se refiere a la audiencia específica que una empresa está tratando de influir o persuadir para lograr sus objetivos comerciales.'

In [ ]:
# import shutil

# # Ruta de la carpeta que deseas comprimir
# folder_path = '/content/vectordb'  # Cambia esto a la ruta de tu carpeta
# zip_path = '/content/vectordb.zip'

# # Comprimir la carpeta
# shutil.make_archive('/content/vectordb', 'zip', folder_path)